# Autonomous Claims Adjudication using Agent Workflows with Strands Agents

## Understanding Multi-Agent Systems and Swarm Intelligence

An agent swarm is a collection of autonomous AI agents working together to solve complex problems through collaboration. Inspired by natural systems like ant colonies or bird flocks, agent swarms leverage collective intelligence where the combined output exceeds what any single agent could produce. By distributing tasks and sharing information, swarms can tackle complex problems more efficiently and effectively than individual agents working in isolation.

Multi-agent systems consist of multiple interacting intelligent agents within an environment. These systems enable:

- *Distributed Problem Solving*: Breaking complex tasks into subtasks for parallel processing
- *Information Sharing*: Agents exchange insights to build collective knowledge
- *Specialization*: Different agents focus on specific aspects of a problem
- *Redundancy*: Multiple agents working on similar tasks improve reliability
- *Emergent Intelligence*: The system exhibits capabilities beyond those of its individual components

Swarm intelligence emphasizes:

1. *Decentralized Control*: No single agent directs the entire system
2. *Local Interactions*: Agents primarily interact with nearby agents
3. *Simple Rules*: Individual agents follow relatively simple behaviors
4. *Emergent Complexity*: Complex system behavior emerges from simple agent interactions

### Autonomous Claims Adjudication

Claims adjudication is the process by which insurance companies process, review, validate, and assess claims to determine if they should be paid, adjusted, or denied.

Here is a Generic Example of a Claim Adjudication Workflow for Auto-insurnace

NOTE: THIS IS JUST AN EXAMPLE FOR ILLUSTRATION PURPOSES. 




<p align="center">
    <img src="./images/ClaimsAdjudicationFlow.png">
</p>

### Workflow Pattern - Sequential

In the Claims Adjudication Flow, there are tasks that have dependencies and   need to be completed before moving to the next task.
We can use a Swarm Sequential pattern where each agent completes its task before passing the result to the next agent in the chain. 


#### Agentic Flow:
An agent that receives an insurance claim (First Notification of Loss), retrieves policy details, validates information against external sources (e.g., repair shop estimates), completes appraisal or damage assessments, and approves payment or flags the claim for human review.

Pattern —> Workflow with Sequential process


<p align="center">
    <img src="./images/ClaimProcessing.png">
</p>

## 1. Quick Start with Swarm tool

The Strands Agents SDK provides a built-in swarm tool that simplifies the implementation of multi-agent systems, offering a quick start for users. This tool implements the shared memory.

In [1]:
!pip install -r requirements.txt
!pip install PyPDF2

  Using cached strands_agents-0.1.9-py3-none-any.whl.metadata (10 kB)
  Using cached strands_agents_tools-0.1.7-py3-none-any.whl.metadata (23 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached mcp-1.10.1-py3-none-any.whl.metadata (40 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached aws_requests_auth-0.4.3-py2.py3-none-any.whl.metadata (567 bytes)
  Using cached slack_bolt-1.23.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached sse_starlette-2.3.6-py3-none-any.whl.metadata (10 kB)
  Using cached slack_sdk-3.35.0-py2.py3-none-any.whl.metadata (15 kB)
Using cached strands_agents-0.1.9-py3-none-any.whl (124 kB)
Using cached strands_agents_tools-0.1.7-py3-none-any.whl (159 kB)
Using cached aws_requests_auth-0.4.3-py2.py3-none-any.whl (6.8 kB)
Using cached docstring_parser-0.16-py3-none-any.whl (36 kB)
Using cached mcp-1.10.1-py3-none-any.whl (150 kB)
Using cached slack_bolt-1.23.0-py2.py3-none-any

In [2]:
from strands import Agent
from strands_tools import swarm
import time
import PyPDF2
import json
import boto3
import yaml
import os
import logging


In [3]:
import PyPDF2

def read_file(file_path):
    """
    Read and extract text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted text from the PDF
    """
    try:
        # Open the PDF file in binary read mode
        # with open(file_path, 'rb') as file:
        #     # Create a PDF reader object
        #     pdf_reader = PyPDF2.PdfReader(file)
        with open(file_path, 'r') as file:
            file = file.read()
            source_json = json.loads(file)
            
            # # Get number of pages
            # num_pages = len(pdf_reader.pages)
            # print(f"Total pages: {num_pages}")
            
            # # Extract text from each page
            # text = ""
            # for page_num in range(num_pages):
            #     page = pdf_reader.pages[page_num]
            #     text += page.extract_text()
                
            return source_json
    
    except FileNotFoundError:
        return "Error: The file was not found."
    except PyPDF2.errors.PdfReadError:
        return "Error: Invalid PDF file or the file is encrypted."
    except Exception as e:
        return f"Error: {str(e)}"



In [4]:
# Example usage
if __name__ == "__main__":
    file_path = "data/FNOL.json"  # Replace with your PDF file path
    fnol_event = read_file(file_path)
    print("fnol_report text:")
    print(fnol_event)

fnol_report text:
{'fnol': {'claimNumber': 'CL-2023-1156789', 'reportDate': '2023-11-21', 'reportTime': '14:35:00', 'reportMethod': 'Online', 'submittedBy': 'Policyholder'}, 'policyInformation': {'policyNumber': 'AUTO-P987654321', 'policyType': 'Auto Insurance', 'effectiveDate': '2023-01-15', 'expirationDate': '2024-01-15', 'insuranceProvider': 'Acme Insurance Company', 'coverageTypes': [{'type': 'Liability', 'limit': '$300,000/$500,000'}, {'type': 'Collision', 'deductible': '$500'}, {'type': 'Comprehensive', 'deductible': '$250'}, {'type': 'Medical Payments', 'limit': '$10,000'}, {'type': 'Uninsured/Underinsured Motorist', 'limit': '$250,000/$500,000'}, {'type': 'Rental Reimbursement', 'limit': '$40/day, $1,200 maximum'}, {'type': 'Roadside Assistance', 'included': True}]}, 'policyholder': {'firstName': 'John', 'lastName': 'Smith', 'dateOfBirth': '1985-06-12', 'address': {'street': '123 Main Street', 'city': 'Springfield', 'state': 'IL', 'zipCode': '62704'}, 'contactInformation': {'ho

In [5]:
from strands import Agent

# Create specialized agents
fnol_processing = Agent(system_prompt=f"""You extract and structure Claim data from FNOL form.""", 
                        callback_handler=None)
appraisal = Agent(system_prompt=f""""You identify damages and assess estimated repair cost based on Claim data.""",
                                 callback_handler=None)
settlement = Agent(system_prompt=f"""You create a Claim payout report.""")

# Sequential workflow processing
def process_workflow(fnol_report):
    # Step 1: FNOL Processing
    fnol_processing_results = fnol_processing(f"process {fnol_event}")

    # Step 2: Damage Assessment
    appraisal_results = appraisal(f"Analyze these research findings: {fnol_processing_results}")

    # Step 3: Payout Generation
    settlement_results = settlement(f"Create a report based on this analysis: {appraisal_results}")

    return settlement_results

In [6]:
process_claim = process_workflow(fnol_event)

# Claim Payout Report

## Claim Summary
**Total Estimated Repair Cost:** $4,500.00

## Damage Assessment
- **Rear bumper:** Significant impact damage requiring replacement
- **Trunk:** Structural damage affecting opening/closing mechanism and alignment
- **Taillights:** Both taillights damaged, requiring replacement
- **Frame:** Possible damage requiring further inspection and potential straightening

## Cost Breakdown
| Item | Cost |
|------|------|
| Rear bumper replacement (parts and labor) | $1,200.00 |
| Trunk repair/alignment | $800.00 |
| Taillight replacement (both sides, parts and labor) | $600.00 |
| Frame inspection and potential repair | $1,500.00 |
| Paint and finish work | $400.00 |

## Additional Costs
| Item | Cost | Coverage |
|------|------|----------|
| Rental vehicle (14 days @ $38/day) | $532.00 | Covered (policy limit: $40/day, $1,200 maximum) |
| Towing | $0.00 | Included in roadside assistance coverage |

## Payout Calculation
- **Total Repair Estimate:** $4,500

### Using the Workflow Tool

In [ ]:
from strands import Agent
from strands_tools import workflow

# Create an agent with workflow capability
agent = Agent(tools=[workflow])

# Create a multi-agent workflow
agent.tool.workflow(
    action="create",
    workflow_id="Claim_adjudication",
    tasks=[
        {
            "task_id": "fnol_processing",
            "description": "Proccess FNOL forms Extract key data to process Claim",
            "system_prompt": f"""Process {fnol_event}You extract and structure Claim data from FNOL form.""",
            "priority": 5
        },
        {
            "task_id": "appraisal",
            "description": "Analyze Claim data, damages and assess estimated repair cost",
            "dependencies": ["fnol_processing"],
            "system_prompt": "You identify damages and assess estimated repair cost based on Claim data.",
            "priority": 3
        },
        {
            "task_id": "settlement",
            "description": "Generate Claim payout",
            "dependencies": ["appraisal"],
            "system_prompt": "You create a Claim payout report.",
            "priority": 2
        }
    ]
)

# Execute workflow (parallel processing where possible)
agent.tool.workflow(action="start", 
                    workflow_id="Claim_processing",
                    )

# Check results
status = agent.tool.workflow(action="status", workflow_id="Claim_adjudication")

Error loading workflow Claim_processing: Expecting value: line 1 column 1 (char 0)
